```{contents}
```
## **Audit Log in LangGraph**

An **Audit Log** in LangGraph is a **tamper-evident, chronological record of all significant events** that occur during graph execution.
It enables **traceability, compliance, debugging, governance, and accountability** for production LLM systems.

---

### **1. Why Audit Logs Are Essential**

LangGraph workflows involve:

* Autonomous decision-making
* Multi-agent collaboration
* External tool execution
* Human-in-the-loop actions
* Long-running stateful processes

This makes **post-hoc accountability mandatory**.

| Requirement   | Why Audit Logs Matter       |
| ------------- | --------------------------- |
| Debugging     | Reconstruct failures        |
| Compliance    | Legal & regulatory evidence |
| Governance    | Enforce policies            |
| Security      | Detect abuse & anomalies    |
| Observability | Understand model behavior   |
| Trust         | Explain decisions           |

---

### **2. What Gets Logged**

An audit log records **structured events**.

| Category        | Examples                             |
| --------------- | ------------------------------------ |
| Graph Lifecycle | Graph compiled, invoked, completed   |
| Execution       | Node start/finish, state diff        |
| Decisions       | Router choices, conditional branches |
| Agents          | Agent messages, role changes         |
| Tools           | Tool calls, inputs, outputs          |
| LLM             | Prompts, responses, tokens           |
| State           | Checkpoints, updates                 |
| Human           | Approvals, corrections               |
| Errors          | Exceptions, retries, recovery        |
| Security        | Auth events, permission checks       |

---

### **3. Conceptual Architecture**

```
Graph Execution
      |
Event Emitter
      |
Structured Log Pipeline
      |
Persistent Audit Store (DB / S3 / SIEM)
      |
Dashboards / Compliance / Forensics
```

---

### **4. Audit Log Data Model**

```json
{
  "timestamp": "2025-01-01T10:15:32Z",
  "run_id": "abc123",
  "thread_id": "user-42",
  "node": "tool_executor",
  "event": "TOOL_CALL",
  "input": {"query": "sales forecast"},
  "output": {"result": "..."},
  "state_diff": {"plan": "updated"},
  "actor": "LLM",
  "status": "SUCCESS",
  "latency_ms": 820
}
```

---

### **5. How LangGraph Emits Audit Events**

LangGraph emits events via:

* **Callbacks**
* **Tracing hooks**
* **State checkpoints**
* **Execution engine notifications**

These can be captured and written into an external audit system.

---

### **6. Implementing Audit Logging**

```python
from langchain.callbacks.base import BaseCallbackHandler

class AuditLogger(BaseCallbackHandler):
    def on_chain_end(self, outputs, **kwargs):
        record_event("CHAIN_END", outputs)

    def on_tool_start(self, serialized, input_str, **kwargs):
        record_event("TOOL_START", input_str)

    def on_tool_end(self, output, **kwargs):
        record_event("TOOL_END", output)
```

```python
graph.invoke(input_data, config={"callbacks": [AuditLogger()]})
```

---

### **7. Production-Grade Audit Store**

| Layer        | Technology                 |
| ------------ | -------------------------- |
| Hot logs     | PostgreSQL / Elasticsearch |
| Cold archive | S3 / GCS                   |
| Security     | Hashing + encryption       |
| Indexing     | Time + run_id + user_id    |
| Retention    | Configurable policies      |

---

### **8. Compliance & Governance Features**

| Feature            | Purpose                |
| ------------------ | ---------------------- |
| Immutable records  | Prevent tampering      |
| Digital signatures | Prove integrity        |
| Access controls    | Limit exposure         |
| Redaction          | Protect sensitive data |
| Retention policies | Regulatory compliance  |

---

### **9. Real-World Example: Regulated Industry**

**Financial AI System**

```
Client → LangGraph → Decision Agents → Execution Tools
                      |
                  Audit Logger
                      |
                Encrypted Audit DB
```

Guarantees:

* Every decision traceable
* Every tool call accountable
* Every human approval recorded

---

### **10. Relationship to Observability**

| Observability | Audit          |
| ------------- | -------------- |
| Performance   | Accountability |
| Debugging     | Compliance     |
| Metrics       | Evidence       |
| Monitoring    | Governance     |

Audit logs are **legal evidence**; observability logs are **engineering tools**.

---

### **11. Best Practices**

* Log **inputs, outputs, and decisions**
* Log **state diffs, not full state**
* Hash & encrypt sensitive content
* Separate audit and application logs
* Test log integrity regularly

---

### **12. Mental Model**

> **Audit Log = Black Box Recorder for AI Systems**

It makes autonomous LLM systems **explainable, governable, and legally defensible**.


### Demonstration

In [2]:
# Demonstration: Production-Style Audit Logging in LangGraph (Single Cell)

from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain_classic.callbacks.base import BaseCallbackHandler
from datetime import datetime
import json

# ---------- Audit Logger ----------

AUDIT_LOG = []

def record_event(event, payload):
    AUDIT_LOG.append({
        "timestamp": datetime.utcnow().isoformat(),
        "event": event,
        "payload": payload
    })

class AuditLogger(BaseCallbackHandler):
    def on_chain_start(self, serialized, inputs, **kwargs):
        record_event("CHAIN_START", inputs)

    def on_chain_end(self, outputs, **kwargs):
        record_event("CHAIN_END", outputs)

    def on_tool_start(self, serialized, input_str, **kwargs):
        record_event("TOOL_START", input_str)

    def on_tool_end(self, output, **kwargs):
        record_event("TOOL_END", output)

# ---------- LangGraph Workflow ----------

class State(TypedDict):
    text: str
    result: str

def processor(state):
    output = state["text"].upper()
    return {"result": output}

builder = StateGraph(State)
builder.add_node("processor", processor)
builder.set_entry_point("processor")
builder.add_edge("processor", END)

graph = builder.compile()

# ---------- Execute with Audit Logging ----------

graph.invoke(
    {"text": "audit logging in langgraph"},
    config={"callbacks": [AuditLogger()]}
)

# ---------- View Audit Log ----------

print(json.dumps(AUDIT_LOG, indent=2))


[
  {
    "timestamp": "2025-12-29T18:07:32.259028",
    "event": "CHAIN_START",
    "payload": {
      "text": "audit logging in langgraph"
    }
  },
  {
    "timestamp": "2025-12-29T18:07:32.260547",
    "event": "CHAIN_START",
    "payload": {
      "text": "audit logging in langgraph"
    }
  },
  {
    "timestamp": "2025-12-29T18:07:32.260547",
    "event": "CHAIN_END",
    "payload": {
      "result": "AUDIT LOGGING IN LANGGRAPH"
    }
  },
  {
    "timestamp": "2025-12-29T18:07:32.261811",
    "event": "CHAIN_END",
    "payload": {
      "text": "audit logging in langgraph",
      "result": "AUDIT LOGGING IN LANGGRAPH"
    }
  }
]


C:\Users\sangouda\AppData\Local\Temp\ipykernel_85428\3656290280.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
